# Segment Anything Model for Geospatial Data 

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/segment-geospatial&urlpath=lab/tree/segment-geospatial/docs/examples/satellite.ipynb&branch=main)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)

This notebook shows how to use segment satellite imagery using the Segment Anything Model (SAM) with a few lines of code. 

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator. 

## Install dependencies

Uncomment and run the following cell to install the required dependencies.


In [2]:
!pip install segment-geospatial leafmap localtileserver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.4 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 19.6 MB/s eta 0:00:00
  Using cached fqdn-1.5.1-py3-none-any.whl (9.1 kB)
  Using cached isoduration-20.11.0-py3-none-any.whl (11 kB)
  Using cached jsonpointer-2.3-py2.py3-none-any.whl (7.8 kB)
  Using cached uri_template-1.2.0-py3-none-any.whl (10 kB)
  Using cached arrow-1.2.3-py3-none-any.whl (66 kB)
  Attempting uninstall: folium
    Found existing installation: folium 0.14.0
    Uninstalling folium-0.14.0:
      Successfully uninstalled folium-0.14.0


## Import libraries

In [3]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

## Create an interactive map

In [4]:
m = leafmap.Map(center=[29.676840, -95.369222], zoom=19)
m.add_basemap("SATELLITE")
m

Map(center=[29.67684, -95.369222], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [5]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]

## Download map tiles

Download maps tiles and mosaic them into a single GeoTIFF file

In [6]:
image = "satellite.tif"

Besides the `satellite` basemap, you can use any of the following basemaps returned by the `get_basemaps()` function:

In [ ]:
# get_basemaps().keys()

Specify the basemap as the source.

In [7]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

Downloaded image 1/40
Downloaded image 2/40
Downloaded image 3/40
Downloaded image 4/40
Downloaded image 5/40
Downloaded image 6/40
Downloaded image 7/40
Downloaded image 8/40
Downloaded image 9/40
Downloaded image 10/40
Downloaded image 11/40
Downloaded image 12/40
Downloaded image 13/40
Downloaded image 14/40
Downloaded image 15/40
Downloaded image 16/40
Downloaded image 17/40
Downloaded image 18/40
Downloaded image 19/40
Downloaded image 20/40
Downloaded image 21/40
Downloaded image 22/40
Downloaded image 23/40
Downloaded image 24/40
Downloaded image 25/40
Downloaded image 26/40
Downloaded image 27/40
Downloaded image 28/40
Downloaded image 29/40
Downloaded image 30/40
Downloaded image 31/40
Downloaded image 32/40
Downloaded image 33/40
Downloaded image 34/40
Downloaded image 35/40
Downloaded image 36/40
Downloaded image 37/40
Downloaded image 38/40
Downloaded image 39/40
Downloaded image 40/40
Saving GeoTIFF. Please wait...
Image saved to satellite.tif


You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [8]:
m.layers[-1].visible = False  # turn off the basemap
m.add_raster(image, layer_name="Image")
m

Map(bottom=27757206.0, center=[29.676820903766707, -95.37017941474916], controls=(ZoomControl(options=['positi…

## Initialize SAM class

In [9]:
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
checkpoint = os.path.join(out_dir, "sam_vit_h_4b8939.pth")

In [19]:
import torch
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]="1"

device = 'mps' if torch.backends.mps.is_available() else 'cpu'

# device = 'cpu'
sam = SamGeo(
    model_type="vit_h",
    checkpoint=checkpoint,
    device=device, 
    sam_kwargs=None,
)

In [20]:
device

'mps'

## Segment the image

Set `batch=True` to segment the image in batches. This is useful for large images that cannot fit in memory.

In [21]:
mask = "segment.tiff"
sam.generate(
    image, mask#, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.

## Polygonize the raster data

Save the segmentation results as a GeoPackage file.

In [ ]:
vector = "segment.gpkg"
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

You can also save the segmentation results as any vector data format supported by GeoPandas.

In [ ]:
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)

## Visualize the results

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector(vector, layer_name="Vector", style=style)
m

![](https://i.imgur.com/Ysq3u7E.png)